In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

import os
import torch
import matplotlib.pyplot as plt
from skimage.io import imread
import torch.nn as nn
import numpy as np
from skimage import img_as_ubyte
import imageio
import json
import cv2
import time
from PIL import Image

import torch.nn.functional as F

from tqdm import tqdm_notebook
# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes
import math
# Data structures and functions for rendering
from pytorch3d.structures import Meshes, Textures
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLPerspectiveCameras, 
    SfMPerspectiveCameras,
    SfMOrthographicCameras,
    PointLights, 
    DirectionalLights,
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    TexturedSoftPhongShader,
    SoftSilhouetteShader,
    look_at_rotation,
    HardFlatShader
)

# add path for demo utils functions 
import sys
import os
import glob
sys.path.append(os.path.abspath(''))

from datetime import datetime
def now_str():
    now = datetime.now()
    month = str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    sec = str(now.second)
    
    output = '[{:>02}/{:>02} {:>02}:{:>02}:{:>02}]'.format(month, day, hour, minute, sec)
    return output
def __output_log(path, strs):
    if not os.path.exists(path):
        with open(path, 'w+') as f:
            f.write(strs)
            f.close()
    else:
        with open(path, 'a+') as f:
            f.write(strs)
            f.close()
print(now_str())
print(torch.__version__)

[04/03 21:34:18]
1.4.0


In [2]:
print('torch.cuda.is_available():',torch.cuda.is_available())

device_gpu = torch.device("cuda:0")
torch.cuda.set_device(device_gpu)
device_cpu = torch.device('cpu')

print('torch.cuda.current_device():', torch.cuda.current_device())
torch.cuda.ipc_collect()
print('torch.cuda.get_device_name(0):',torch.cuda.get_device_name(0))

# print('GPU memory stats ---------------------')
# gpu_mem_stats = torch.cuda.memory_stats(device=device_gpu)
# for k, v in gpu_mem_stats.items():
#     print('  {}: {}'.format(k, v))

print(torch.cuda.memory_summary(device=device_gpu, abbreviated=False))
bytes_reserved = torch.cuda.memory_reserved()
print('torch.cuda.memory_reserved(): {:,.2f} Mb'.format(bytes_reserved * 0.000001))
# Returns the current GPU memory usage by 
# tensors in bytes for a given device
bytes_alloc = torch.cuda.memory_allocated()
print('torch.cuda.memory_allocated(): {:,.2f} Mb'.format(bytes_alloc * 0.000001))

# Returns the current GPU memory managed by the
# caching allocator in bytes for a given device
bytes_cached = torch.cuda.memory_cached()
print('torch.cuda.memory_cached(): {:,.2f} Mb'.format(bytes_cached * 0.000001))



# Releases all unoccupied cached memory currently held by
# the caching allocator so that those can be used in other
# GPU application and visible in nvidia-smi
print('----- torch.cuda.empty_cache() -----')

torch.cuda.empty_cache()
bytes_reserved = torch.cuda.memory_reserved()
print('torch.cuda.memory_reserved(): {:,.2f} Mb'.format(bytes_reserved * 0.000001))
bytes_alloc = torch.cuda.memory_allocated()
print('torch.cuda.memory_allocated(): {:,.2f} Mb'.format(bytes_alloc * 0.000001))
bytes_cached = torch.cuda.memory_cached()
print('torch.cuda.memory_cached(): {:,.2f} Mb'.format(bytes_cached * 0.000001))

torch.cuda.is_available(): True
torch.cuda.current_device(): 0
torch.cuda.get_device_name(0): GeForce RTX 2070 SUPER
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |

In [3]:
import math
def Rz(r):
    # r = rad
    R = np.float32([[math.cos(r), -math.sin(r), 0], [math.sin(r), math.cos(r), 0], [0, 0, 1]])
    return R

In [4]:
# R = torch.from_numpy(np.identity(3).astype(np.float32)).unsqueeze(0)
R_z = Rz(-np.pi/2)
R = torch.from_numpy(R_z).unsqueeze(0)
T = torch.from_numpy(np.array([0, 0, 0]).astype(np.float32)).unsqueeze(0)
print(R_z)

[[ 6.123234e-17  1.000000e+00  0.000000e+00]
 [-1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]


In [5]:
p1 = np.float32([10, 0, 0])
p2 = np.float32([0, 10, 0])
p1b = R_z.dot(p1)
p2b = R_z.dot(p2)
p1b_column_major = p1.dot(R_z)
p2b_column_major = p2.dot(R_z)
print('if R is ext:              ', p1b)
print('if R is ext, column_major:', p1b_column_major)
print()
print('if R is ext:              ', p2b)
print('if R is ext, column_major:', p2b_column_major)
ps = np.array([p1, p2])

if R is ext:               [ 6.123234e-16 -1.000000e+01  0.000000e+00]
if R is ext, column_major: [6.123234e-16 1.000000e+01 0.000000e+00]

if R is ext:               [1.000000e+01 6.123234e-16 0.000000e+00]
if R is ext, column_major: [-1.000000e+01  6.123234e-16  0.000000e+00]


In [7]:
principal_point = torch.from_numpy(np.array([1, 1]).astype(np.float32)).unsqueeze(0)
focal_length = torch.from_numpy(np.array([0, 0]).astype(np.float32)).unsqueeze(0)
camera = SfMPerspectiveCameras(device=device_cpu, R=R, T=T, principal_point=principal_point, focal_length=focal_length)
E = camera.get_world_to_view_transform()
psb = E.transform_points(torch.from_numpy(ps).unsqueeze(0))
print(psb.numpy())

  composed_matrix:
[[ 6.123234e-17 -1.000000e+00  0.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
[[[ 6.123234e-16 -1.000000e+01  0.000000e+00]
  [ 1.000000e+01  6.123234e-16  0.000000e+00]]]


In [8]:
# verts_world = torch.from_numpy(np.array([[-354.0330,  207.6380, 1707.0809], [-356.3820,  195.7520, 1695.5021]]).astype(np.float32)).unsqueeze(0)
verts_world = torch.from_numpy(np.array([[100, 0, 0], [0, 0, 0]]).astype(np.float32)).unsqueeze(0)
verts_screen = camera.transform_points(verts_world)
print(verts_screen)

1 transform_points. KE=
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 1. 1. 0.]]
  composed_matrix:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 1. 1. 0.]]
tensor([[[inf, inf, inf],
         [inf, inf, inf]]])


In [ ]:
def reproj_pytorch3d(camera, verts_world):
    E = camera.get_world_to_view_transform().get_matrix()[0]

    Kmtx = camera.get_projection_transform().get_matrix()[0]
    fx = Kmtx[0, 0]
    fy = Kmtx[1, 1]
    px = Kmtx[3, 0]
    py = Kmtx[3, 1]
    
    K = np.array([[fx,   0,    0,  0], [0,   fy,    0,  0], [0,    0,    0,   1], [px, py, 1, 0]])
    print(K)
    pts = verts_world.squeeze().numpy()
    print(pts.shape)
    for i in range(pts.shape[0]):
        Vw = np.array([pts[i, 0], pts[i, 1], pts[i, 2], 1]).astype(np.float32)
        
        # extrinsics
        Vc = Vw.dot(E)
        
        # intrinsics
        U = Vc.dot(K)
        U[0] /= U[3]
        U[1] /= U[3]
        
        print(U)
reproj_pytorch3d(camera, verts_world)    